# SQLAlchemy Data Model from Reflection

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import mapper, relationship
from sqlalchemy import inspect, Table, MetaData, Column, Integer, String, ForeignKey
from sqlalchemy import text

In [ ]:
pg_driver = "postgresql+psycopg2://"
pg_uri = "user=tester password=password host=localhost port=5432 dbname=pagila"
pg_schema = 'public'
def pg_creator():
    return psycopg2.connect(pg_uri)
    #return psycopg2.connect(user="tester", password="password", host="localhost", port="5432", database="pagila")
pg_engine = sqlalchemy.create_engine(pg_driver, creator=pg_creator)
pg_Base = automap_base()
pg_Base.prepare(pg_engine, schema=pg_schema, reflect=True)
pg_session = Session(pg_engine)

In [ ]:
metadata = MetaData()
metadata.reflect(bind=pg_engine)

In [ ]:
list_tables = ['country', 'city', 'address', 'store', 'customer']

In [ ]:
MAPPERS = {}
repr_name = lambda t: '%s%s' % (t[0].upper(), t[1:]) # change the first charactor of the string to upper case

In [ ]:
for table_name in list_tables:
    table = metadata.tables[table_name]
    cls = None
    # 1. create class object
    cls_name = repr_name(str(table))
    print(f"create class {cls_name}")
    exec("""class %s(object): pass""" % cls_name)
    exec("""cls = %s""" % cls_name)

    mapper(cls, table, properties={})
    MAPPERS.update({cls_name: cls})
    '''
    # 2. collect relations by FK
    properties = {}
    #for c in metadata.tables[table].columns:
    for c in table.columns:
        for fk in c.foreign_keys:
            name = str(fk.column).split('.')[0]
            print(f"    foreign key column {name} = {MAPPERS[repr_name(name)]}")
            properties.update({
                name: relationship(lambda: MAPPERS[repr_name(name)]),
            })

    print(f"    foreign key properties {properties}")
    # 3. map table to class object 
    mapper(cls, table, properties=properties)

    MAPPERS.update({cls_name: cls})
    '''

In [ ]:
MAPPERS

In [ ]:
insp = inspect(Address)
list(insp.columns)

In [ ]:
insp = inspect(City)
list(insp.columns)

In [ ]:
table = metadata.tables['address']

In [ ]:
list(table.columns.city_id.foreign_keys)[0]._colspec

In [ ]:
data = pg_session.query(Address).filter_by(city_id=1)

In [ ]:
tables = dict()
for t in list_tables:
    table_class = MAPPERS[repr_name(t)]
    data = pg_session.query(table_class)
    insp = inspect(table_class)
    tables[t] = (data, insp)
    #for rec in data:
    #    print("\t".join([str(getattr(rec, field)) for field in insp.column_attrs.keys()]))
    

In [ ]:
table_name = "customer"
for rec in tables[table_name][0]:
    print("\t".join([str(getattr(rec, field)) for field in tables[table_name][1].column_attrs.keys()]))

In [ ]:
customer_ret = pg_session.query(Customer).filter_by(first_name='JOYCE')
vars(customer_ret[0])

In [ ]:
store_ret = pg_session.query(Store).filter_by(store_id=customer_ret[0].store_id)
vars(store_ret[0])

In [ ]:
address_ret = pg_session.query(Address).filter_by(address_id=store_ret[0].address_id)
vars(address_ret[0])

In [ ]:
city_ret = pg_session.query(City).filter_by(city_id=address_ret[0].city_id)
vars(city_ret[0])

In [ ]:
country_ret = pg_session.query(Country).filter_by(country_id=city_ret[0].country_id)
vars(country_ret[0])

# Raw SQL

In [ ]:
# engine level reflection
inspector = inspect(pg_engine)

In [ ]:
inspector.get_columns('payment')

In [ ]:
select_statement = text("""
    select p.payment_id, 
           concat(cu.first_name, ' ', cu.last_name) "name" 
    from payment p, customer cu 
    where cu.customer_id = p.customer_id 
    order by p.payment_id desc 
    limit 10""")

### select

In [ ]:
data = list()
with pg_engine.connect() as con:
    rs = con.execute(select_statement)
    table_fields = rs.keys()
    for row in rs:
        row_dict = dict(zip(rs.keys(), row))
        data.append(row_dict)
print(table_fields)
for _ in data:
    print(_)

### insert

In [ ]:
select_statement = text("""select * from payment order by payment_id desc limit 1""")
data = list()
with pg_engine.connect() as con:
    rs = con.execute(select_statement)
    table_fields = rs.keys()
    for row in rs:
        row_dict = dict(zip(rs.keys(), row))
        data.append(row_dict)
print("select:", table_fields, data)

with pg_engine.connect() as con:
    insert_statement = text(f"INSERT INTO payment(customer_id, staff_id, rental_id, amount, payment_date) VALUES(:customer_id, :staff_id, :rental_id, :amount, :payment_date)")
    for line in data:
        ret = con.execute(insert_statement, **line)

### delete

In [ ]:
with pg_engine.connect() as con:
    delete_statement = text("""delete from payment where payment_id = (select max(payment_id) from payment)""")
    ret = con.execute(delete_statement)